In [ ]:
from setup_general import *
from setup_embedding import *

In [ ]:
data = curie.copy()
data.drop(columns=['text_features'], inplace=True)
data.set_index('id', inplace=True)
split = pd.DataFrame(data.curie_similarity.tolist(), index=data.index)
concat = pd.concat([data, split], axis=1)
concat.drop(columns=['curie_similarity'], inplace=True)
concat.to_csv('data/text_embeddings/curie.csv')
curie = pd.read_csv('data/text_embeddings/curie.csv', index_col='id', dtype={'type': str})
train = curie[curie.source == 'train']
test = curie[curie.source == 'test']
train.drop(columns=['source'], inplace=True)
test.drop(columns=['source'], inplace=True)
train.to_csv('data/text_embeddings/train_curie.csv')
test.to_csv('data/text_embeddings/test_curie.csv')